In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import random
from sklearn.metrics import classification_report

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Khám phá các cột của dữ liệu

In [ ]:
with open('../input/home-credit-default-risk/HomeCredit_columns_description.csv', 'r', newline='', encoding='ISO-8859-1') as csvfile:
    df = pd.read_csv(csvfile)
df = df.set_index('Unnamed: 0')
application_columns = df[df['Table'] == 'application_{train|test}.csv']
print(len(application_columns.index))
application_columns

In [ ]:
bureau_columns = df[df['Table'] == 'bureau.csv']
print(len(bureau_columns.index))
bureau_columns

In [ ]:
bureau_balance_column = df[df['Table'] == 'bureau_balance.csv']
print(len(bureau_balance_column.index))
bureau_balance_column

In [ ]:
POS_CASH_balance_column = df[df['Table'] == 'POS_CASH_balance.csv']
print(len(POS_CASH_balance_column.index))
POS_CASH_balance_column

In [ ]:
credit_card_balance_column = df[df['Table'] == 'credit_card_balance.csv']
print(len(credit_card_balance_column.index))
credit_card_balance_column

In [ ]:
previous_application_column = df[df['Table'] == 'previous_application.csv']
print(len(previous_application_column.index))
previous_application_column

In [ ]:
installments_payments_column = df[df['Table'] == 'installments_payments.csv']
print(len(installments_payments_column.index))
installments_payments_column

In [ ]:
df_app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv', index_col='SK_ID_CURR')

In [ ]:
# df_app_train['SK_ID_CURR'].dtype == np.int64
df_app_train.head()

# Xử lý các cột Categorical

In [ ]:
print('Các cột categorical')
categorical_columns = list(set(df_app_train.columns) - set(df_app_train._get_numeric_data().columns))
print(categorical_columns)

In [ ]:
x = [len(df_app_train[c].unique()) for c in categorical_columns]
x = list(zip(x, categorical_columns))
x = pd.DataFrame(data=sorted(x, reverse=True), columns=['Values', 'Name'])

plt.figure(figsize=(16,10), dpi= 60)
sns.barplot(data=x, x='Name', y='Values')
plt.title('Thống kê số lượng giá trị khác nhau của các cột Categorical')

plt.xticks(rotation=45)
plt.show()

# Missing Value

In [ ]:
print(df_app_train.info())

# Chạy thử mô hình đơn giản

In [ ]:
numeric_data = set(df_app_train._get_numeric_data().columns)
simple_df = df_app_train[numeric_data]

In [ ]:
df_data = simple_df.drop('TARGET', axis=1)
X = df_data.to_numpy()
y = simple_df['TARGET'].to_numpy()
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True)

In [ ]:
clf = xgb.XGBClassifier(
        n_estimators=5000,
        max_depth=12,
        learning_rate=0.02,
        subsample=0.8,
        colsample_bytree=0.4,
        missing=-1,
        eval_metric='auc',
        # USE CPU
#         nthread=4,
        #tree_method='hist'
        # USE GPU
        tree_method='gpu_hist' 
    )        
h = clf.fit(X_train, y_train, 
        eval_set=[(X_val,y_val)],
        verbose=100, early_stopping_rounds=200)

In [ ]:
y_pred = h.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))